# Load ENV File

In [1]:
from pathlib import Path

from src.custom_types import EnvironmentType

from dotenv import load_dotenv

dotenv_path = Path("envs/.env")
load_dotenv(dotenv_path=dotenv_path)
print(f"Loaded {dotenv_path}")

Loaded envs\.env


# Create DB_MANAGER

In [2]:
from src.db.db_manager import DBManager
from src.utils import get_db_settings

db_manager = DBManager(
    db_settings=get_db_settings(),
)

DBManager created.


# EXAMPLES

## Create Moneyboxes

In [5]:
import uuid

GENERATE_N_MONEYBOXES = 10
created_moneyboxes = {}


moneybox_data = {"name": "Test"}
moneybox = await db_manager.add_moneybox(moneybox_data=moneybox_data)

for _ in range(GENERATE_N_MONEYBOXES):
    moneybox_data = {"name": f"Test Moneybox [{uuid.uuid4()}]"}
    moneybox = await db_manager.add_moneybox(moneybox_data=moneybox_data)
    created_moneyboxes[moneybox["id"]] = moneybox
    print(f"Created: {moneybox}")

IntegrityError: (sqlalchemy.dialects.postgresql.asyncpg.IntegrityError) <class 'asyncpg.exceptions.UniqueViolationError'>: duplicate key value violates unique constraint "ix_moneybox_name"
DETAIL:  Key (lower(name::text))=(test) already exists.
[SQL: INSERT INTO moneybox (name, balance) VALUES ($1::VARCHAR, $2::INTEGER) RETURNING moneybox.name, moneybox.balance, moneybox.id]
[parameters: ('Test', 0)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

## Update Moneybox 1

In [9]:
import uuid

moneybox_data = {"name": f"Test Moneybox 42"}
moneybox = await db_manager.update_moneybox(moneybox_id=1, moneybox_data=moneybox_data)

print(f"Updated Moneybox: {moneybox}")

Updated Moneybox: {'name': 'Test Moneybox 42', 'balance': 0, 'id': 1}


## Select Moneybox 1

In [7]:
import uuid

moneybox = await db_manager.get_moneybox(moneybox_id=1)

print(f"Moneybox: {moneybox}")

Moneybox: {'name': 'TEST 4242', 'balance': 0, 'id': 1}


## Delete Moneyboxes

In [ ]:
from pprint import pprint

# pprint(created_moneyboxes)

delete_moneybox_ids = [
    moneybox_id for moneybox_id, moneybox in created_moneyboxes.items() if moneybox["id"] % 3 == 0
]
print(f"Try to delete Moneyboxex with id: {', '.join(map(str, delete_moneybox_ids))}")


for moneybox_id in delete_moneybox_ids:
    try:
        await db_manager.delete_moneybox(moneybox_id=moneybox_id)
    except Exception as ex:
        print(ex)